# QCIntegrals

In [30]:
using HypergeometricFunctions
using SpecialFunctions
using LinearAlgebra
using BenchmarkTools

In [31]:
function doublefactorial(n)
    if (n <= 1)
        return 1
    end
    return n * doublefactorial(n - 2)
end

doublefactorial (generic function with 1 method)

In [32]:
function F_n(n,x)
    if(x!=0)
        return gamma(0.5 + n) * gamma_inc(0.5 + n, x, 0)[1] / (2*x^(0.5 + n))
    else
        return HypergeometricFunctions.M(n+1/2,n+3/2,-x)/(2*n+1)
    end
end

F_n (generic function with 1 method)

## Shell

In [33]:
struct shell
    l::Int8
    exp::Vector{Float64}
    coef::Vector{Float64}
    coord::Vector{Float64}
    N::Vector{Float64}
    orientaciones::Vector{Vector{Int}}
end

function build_shell(l,exp,coef,coord;normalized=true)

    #normalizacion
    for i in eachindex(exp)
        coef[i] = coef[i]*(2*exp[i]/pi)^(3/4)*(4*exp[i])^(l/2)
    end
    
    norm = 0
    for j in eachindex(exp)
        for k in eachindex(exp)
            norm += coef[j]*coef[k]*(pi/(exp[j]+exp[k]))^(3/2)/(2*(exp[j]+exp[k]))^(l)
        end
    end

    for i in eachindex(exp)
        coef[i] = coef[i]/sqrt(norm)
    end
        
    orientaciones = []
    for lx in l:-1:0
        for ly in l-lx:-1:0
            lz = l-lx-ly
            append!(orientaciones,[[lx,ly,lz]])      
        end
    end        

    N = ones(Int((l+1)*(l+2)/2))
    if normalized
        for (i,(lx,ly,lz)) in enumerate(orientaciones)
            N[i] = sqrt(1/(doublefactorial(2*lx-1)*doublefactorial(2*ly-1)*doublefactorial(2*lz-1)))
        end
    end
    
    return shell(l, exp, coef, coord, N, orientaciones)
    
end

function build_zero_shell()

    #normalizacion
    l = 0
    exp = [0.0]
    coef = [1.0]
    coord = [0.0,0.0,0.0]
    N = [1.0]
    orientaciones = [[0,0,0]]
    
    return shell(l, exp, coef, coord, N, orientaciones)
    
end

build_zero_shell (generic function with 1 method)

In [34]:
g_a = build_shell(0,[13.01, 1.962, 0.4446, 0.122],[0.01968500,0.1379770,0.4781480,0.5012400],[0.0,0.0,0.0])
g_b = build_shell(1,[0.07896],[1.0],[0,0,0])
g_c = build_shell(0,[0.141],[1.0],[0.6068,-0.2383,-0.7169])
g_d = build_shell(0,[0.07896],[1.0],[0,0,0])

shell(0, [0.07896], [0.10616110906976056], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])

## Shell Pair

In [35]:
struct shell_pair
    
    g_a::shell
    g_b::shell
    
    exp::Vector{Float64}
    coef::Vector{Float64}
    coord::Vector{Vector{Float64}}
    alpha::Vector{Float64}
    
end
    
function build_shell_pair(g_a, g_b)
    
    exp = vec((g_a.exp .+ g_b.exp')')
    coef = vec((g_a.coef .* g_b.coef')')

    coord = []
    for i in eachindex(g_a.exp)
        for j in eachindex(g_b.exp)
            append!(coord,[(g_a.exp[i]*g_a.coord+g_b.exp[j]*g_b.coord)/(g_a.exp[i]+g_b.exp[j])])
        end
    end
            
    alpha = vec(((g_a.exp .* g_b.exp')./(g_a.exp .+ g_b.exp'))')
    
    return shell_pair(g_a, g_b, exp, coef, coord, alpha)

end

build_shell_pair (generic function with 1 method)

In [36]:
pairAB = build_shell_pair(g_a,g_b)
pairCD = build_shell_pair(g_c,g_d)

shell_pair(shell(0, [0.141], [0.163992666204993], [0.6068, -0.2383, -0.7169], [1.0], [[0, 0, 0]]), shell(0, [0.07896], [0.10616110906976056], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]]), [0.21996], [0.017409643323629098], [[0.38897435897435895, -0.15275641025641026, -0.45955128205128204]], [0.05061538461538462])

## Creador de Shell

In [37]:
function build_set_of_shells(element,coord,basis_name;normalized=true,auxiliar=false)

    key_to_l = Dict("S" => 0, "P" => 1, "D" => 2, "F" => 3, "G" => 4, "H" => 5, "I" => 6)

    element = uppercase(element)
    shells = []
    
    # Basis file
    #filename = joinpath(@__DIR__, "../basis/"*basis_name*".gbs")
    #f = open(filename,"r")
    f = open("basis/"*basis_name*".gbs","r")
    lines = readlines(f)
    close(f)
    
    # Find_atom
    atom_line = 0
    for (i,line) in enumerate(lines)
        splitted_line = split(line)
        if(size(splitted_line)[1]>0)
            if(occursin("****",splitted_line[1]))
                next_line = split(lines[i+1])
                if(size(next_line)[1]>0)        
                    if(next_line[1] == element)
                        atom_line = i+1
                        break
                    end
                end
            end
        end
    end

    # Read shells
    num_of_primitives = 0
    l = 0
    exp = []
    coef = []
    for line in lines[atom_line+1:size(lines)[1]]
        splitted_line = split(line)
        if(size(splitted_line)[1]==3)
            exp = []
            coef = []
            try
                l = parse(Int,splitted_line[1])
            catch
                l = key_to_l[splitted_line[1]]
            end
            num_of_primitives = parse(Int,splitted_line[2])
        elseif(size(splitted_line)[1]==2)
            append!(exp,parse(Float64,splitted_line[1]))           
            append!(coef,parse(Float64,splitted_line[2]))
            if(size(exp)[1] == num_of_primitives)
                if(auxiliar)
                    shell = build_aux_shell(l,exp,coef,coord)
                else
                    shell = build_shell(l,exp,coef,coord,normalized=normalized)
                end
                push!(shells,shell)
            end
        end
        if(occursin("****",splitted_line[1]))
            break
        end
    end
            
    return shells
                        
end

build_set_of_shells (generic function with 1 method)

In [38]:
shells = build_set_of_shells("O",[0.0,0.0,0.0],"aug-cc-pvdz",normalized=true)

9-element Vector{Any}:
 shell(0, [11720.0, 1759.0, 400.8, 113.7, 37.03, 13.27, 5.025, 1.013, 0.3023], [0.569986149793314, 1.058880501673631, 1.7771678966453877, 2.600709693959018, 3.028357204192797, 2.223503905508639, 0.6481188547796517, 0.011124247231272932, -0.0007511028383196536], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])
 shell(0, [11720.0, 1759.0, 400.8, 113.7, 37.03, 13.27, 5.025, 1.013, 0.3023], [-0.12844755012231768, -0.24449099282587303, -0.40009729327831506, -0.6381663487030188, -0.7587848290723769, -0.8196486224662376, -0.27975700946278415, 0.4011060836621272, 0.1664219730305194], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])
 shell(0, [0.3023], [0.2905619240088511], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])
 shell(0, [0.07896], [0.10616110906976056], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])
 shell(1, [17.7, 3.854, 1.046, 0.2753], [2.226162227432104, 1.7619764776359819, 0.7670792917006999, 0.13090512359254836], [0.0, 0.0, 0.0], [1.0, 1.0, 1.0], [[1, 0, 0], [0, 1, 0], [0, 0, 1]])
 shell(1, [0.2753]

In [47]:
struct molecule
    charge
    mult
    symbols
    Zs
    coords
end

function build_molecule(xyz;units="Angstrom")
    
    charge = 0
    mult = 1
    
    symbols = []
    Zs = []
    coords = []
    
    symbol_to_Z = Dict("H" => 1, "He" => 2, "Li" => 3,
        "Be" => 4, "B" => 5, "C" => 6, "N" => 7, "O" => 8, "F" => 9, "Ne" => 10,
        "Na" => 11, "Mg" => 12, "Al" => 13, "Si" => 14, "P" => 15,"S" => 16, "Cl" => 17, "Ar" => 18,
        "K" => 19, "Ca" => 20, "Sc" => 21, "Ti" => 22, "V" => 23,"Cr" => 24, "Mn" => 25, "Fe" => 26, "Co" => 27, "Ni" => 28, "Cu" => 29, "Zn" => 30, "Ga" => 31,"Ge" => 32, "As" => 33, "Se" => 34, "Br" => 35, "Kr" => 36,
        "Rb" => 37, "Sr" => 38, "Y" => 39, "Zr" => 40, "Nb" => 41,"Mo" => 42, "Tc" => 43, "Ru" => 44, "Rh" => 45, "Pd" => 46, "Ag" => 47, "Cd" => 48, "In" => 49,"Sn" => 50, "Sb" => 51, "Te" => 52, "I" => 53, "Xe" => 54,)     
    
    xyz = split(xyz,"\n")    
    natoms = size(xyz)[1]
    for iatom in 1:natoms
        line = split(xyz[iatom])
        if size(line)[1] == 4
            append!(symbols,line[1])
            append!(Zs,symbol_to_Z[line[1]])
            append!(coords,[[parse(Float64,line[2]),parse(Float64,line[3]),parse(Float64,line[4])]])
        elseif size(line)[1] == 2
            charge = parse(Float64,line[1])
            mult = parse(Float64,line[2])
        end
    end
    
    if(units=="Angstrom")
        coords = coords*1.88973
    end
    
    return molecule(charge,mult,symbols,Zs,coords)

end

function Ne(mol)
    return sum(mol.Zs) - mol.charge
end

function Nalpha(mol)
    return (sum(mol.Zs) - mol.charge + (mol.mult-1))/2
end

function Nbeta(mol)
    return (sum(mol.Zs) - mol.charge - (mol.mult-1))/2
end

Nbeta (generic function with 1 method)

In [48]:
mol = build_molecule("""
O   0.000000  0.000000  0.000000
H   0.277400  0.892900  0.254400
H   0.606800 -0.238300 -0.716900
""")

molecule(0, 1, Any['O', 'H', 'H'], Any[8, 1, 1], [[0.0, 0.0, 0.0], [0.524211102, 1.687339917, 0.480747312], [1.146688164, -0.450322659, -1.3547474369999999]])

In [49]:
function build_basis(mol,basis_name;normalized=true,auxiliar=false)
    shells = []
    for (symbol,coord) in zip(mol.symbols,mol.coords)
        shell = build_set_of_shells(string(symbol),coord,basis_name,normalized=normalized,auxiliar=auxiliar)
        append!(shells,shell)
    end
    return shells        
end

build_basis (generic function with 1 method)

In [50]:
shells = build_basis(mol,"aug-cc-pvdz")

19-element Vector{Any}:
 shell(0, [11720.0, 1759.0, 400.8, 113.7, 37.03, 13.27, 5.025, 1.013, 0.3023], [0.569986149793314, 1.058880501673631, 1.7771678966453877, 2.600709693959018, 3.028357204192797, 2.223503905508639, 0.6481188547796517, 0.011124247231272932, -0.0007511028383196536], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])
 shell(0, [11720.0, 1759.0, 400.8, 113.7, 37.03, 13.27, 5.025, 1.013, 0.3023], [-0.12844755012231768, -0.24449099282587303, -0.40009729327831506, -0.6381663487030188, -0.7587848290723769, -0.8196486224662376, -0.27975700946278415, 0.4011060836621272, 0.1664219730305194], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])
 shell(0, [0.3023], [0.2905619240088511], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])
 shell(0, [0.07896], [0.10616110906976056], [0.0, 0.0, 0.0], [1.0], [[0, 0, 0]])
 shell(1, [17.7, 3.854, 1.046, 0.2753], [2.226162227432104, 1.7619764776359819, 0.7670792917006999, 0.13090512359254836], [0.0, 0.0, 0.0], [1.0, 1.0, 1.0], [[1, 0, 0], [0, 1, 0], [0, 0, 1]])
 shell(1, [0.2753

In [14]:
function get_nbf(shells)
    
    nbf = 0
    
    for shell in shells
        nbf += size(shell.orientaciones)[1]
    end
    
    return nbf
    
end

get_nbf (generic function with 1 method)

In [15]:
nbf = get_nbf(shells)

0

## MD

In [16]:
function build_R(lt,alpha,Rpq,F,pairAB,pairCD)
    R = [[[[[[0.0 for lz in 0:lt-n-lx-ly] for ly in 0:lt-n-lx] for lx in 0:lt-n] for n in 0:lt] for jj in eachindex(pairCD.exp)] for ii in eachindex(pairAB.exp)]
    for ii in eachindex(pairAB.exp)
        for jj in eachindex(pairCD.exp)
            for nini in 0:lt
                R[ii][jj][nini+1][1][1][1] = (-2*alpha[ii][jj])^nini*2*pi^(5/2)/(pairAB.exp[ii]*pairCD.exp[jj]*sqrt(pairAB.exp[ii]+pairCD.exp[jj]))*F[ii][jj][nini+1]
                l = 0
                for n in nini-1:-1:0
                    l += 1
                    for lx in 0:l
                        for ly in 0:l-lx
                            lz = l-lx-ly
                            if(lz>=2)
                                R[ii][jj][n+1][lx+1][ly+1][lz+1] += (lz-1)*R[ii][jj][n+2][lx+1][ly+1][lz-1]
                            end
                            if(lz>=1)
                                R[ii][jj][n+1][lx+1][ly+1][lz+1] += Rpq[ii][jj][3]*R[ii][jj][n+2][lx+1][ly+1][lz]
                            end
                        end
                        ly = l-lx
                        if(ly>=2)
                            R[ii][jj][n+1][lx+1][l-lx+1][1] += (ly-1)*R[ii][jj][n+2][lx+1][l-lx-1][1]
                        end
                        if(ly>=1)
                            R[ii][jj][n+1][lx+1][l-lx+1][1] += Rpq[ii][jj][2]*R[ii][jj][n+2][lx+1][l-lx][1]
                        end
                    end
                    lx = l
                    if(lx>=2)
                        R[ii][jj][n+1][l+1][1][1] += (lx-1)*R[ii][jj][n+2][l-1][1][1]
                    end
                    if(lx>=1)
                        R[ii][jj][n+1][l+1][1][1] += Rpq[ii][jj][1]*R[ii][jj][n+2][l][1][1]                
                    end
                end
            end
        end
    end
             
    return R

end

build_R (generic function with 1 method)

In [17]:
function build_E(l1,l2,pair)
    E = [[[[[0.0 for t in 0:la+lb] for lb in 0:l2] for la in 0:l1] for xyz in 1:3] for ii in eachindex(pair.exp)]

    for ii in eachindex(pair.exp)
        for xyz in 1:3
            E[ii][xyz][1][1][1] = exp(-pair.alpha[ii]*(pair.g_b.coord[xyz]-pair.g_a.coord[xyz])^2)
            for la in 0:l1
                for lb in 0:l2
                    for t in 0:la+lb
                        if(lb>0)
                            if(t>0)               
                                E[ii][xyz][la+1][lb+1][t+1] += 1/(2*pair.exp[ii])*E[ii][xyz][la+1][lb][t]
                            end
                            if(t<=la+lb-1)
                                E[ii][xyz][la+1][lb+1][t+1] += (pair.coord[ii][xyz]-pair.g_b.coord[xyz])*E[ii][xyz][la+1][lb][t+1]
                            end
                            if(t+1<=la+lb-1)
                                E[ii][xyz][la+1][lb+1][t+1] += (t+1)*E[ii][xyz][la+1][lb][t+2]
                            end
                        elseif(la>0)
                            if(t>0)               
                                E[ii][xyz][la+1][lb+1][t+1] += 1/(2*pair.exp[ii])*E[ii][xyz][la][lb+1][t]
                            end
                            if(t<=la-1+lb)
                                E[ii][xyz][la+1][lb+1][t+1] += (pair.coord[ii][xyz]-pair.g_a.coord[xyz])*E[ii][xyz][la][lb+1][t+1]
                            end
                            if(t+1<=la-1+lb)
                                E[ii][xyz][la+1][lb+1][t+1] += (t+1)*E[ii][xyz][la][lb+1][t+2]
                            end
                        end
                    end
                end
            end
        end
    end
                                
    return E
    
end

build_E (generic function with 1 method)

## Overlap

In [18]:
function overlap(pairAB)
    
    g_a = pairAB.g_a
    g_b = pairAB.g_b

    Eab = build_E(g_a.l,g_b.l,pairAB)
    result = zeros(size(g_a.orientaciones)[1],size(g_b.orientaciones)[1])

    for (a,(lax,lay,laz)) in enumerate(g_a.orientaciones)
        for (b,(lbx,lby,lbz)) in enumerate(g_b.orientaciones)
        
            integral = 0.0
            for ii in eachindex(pairAB.exp)
                t_z = 0
                t_y = 0
                t_x = 0
                integral += Eab[ii][1][lax+1][lbx+1][t_x+1]*
                            Eab[ii][2][lay+1][lby+1][t_y+1]*
                            Eab[ii][3][laz+1][lbz+1][t_z+1]*
                            (pi/pairAB.exp[ii])^1.5*pairAB.coef[ii]
            end           
            result[a,b] = g_a.N[a]*g_b.N[b]*integral
        end
    end
    
    return result
    
end

overlap (generic function with 1 method)

In [19]:
function build_S(shells)
    
    nbf = get_nbf(shells)
    
    S = zeros(nbf,nbf)
    i = 1
    for g_a in shells
        j = 1
        for g_b in shells
            pair = build_shell_pair(g_a,g_b)
            results = overlap(pair)
            S[i:i+size(g_a.orientaciones)[1]-1,j:j+size(g_b.orientaciones)[1]-1] = results
            j += size(g_b.orientaciones)[1]
        end
        i += size(g_a.orientaciones)[1]
    end
    
    return S
    
end

build_S (generic function with 1 method)

In [20]:
@btime S = build_S(shells)

  47.640 ns (1 allocation: 64 bytes)


0×0 Matrix{Float64}

# Cinética

In [21]:
function Tij(la,lb,xyz,pairAB)
    
    Eab = build_E(pairAB.g_a.l,pairAB.g_b.l,pairAB)
    
    S = [[[0.0 for j in 0:lb] for i in 0:la] for ii in eachindex(pairAB.exp)]
    T = [[[0.0 for j in 0:lb] for i in 0:la] for ii in eachindex(pairAB.exp)]
    
    for ii in eachindex(pairAB.exp)
        i = 0
        t = 0
        for j in 0:lb
            S[ii][i+1][j+1] = Eab[ii][xyz][i+1][j+1][t+1]*(pi/pairAB.exp[ii])^0.5            
        end
        for i in 1:la
            for j in max(0,lb-la+i):lb
                S[ii][i+1][j+1] = Eab[ii][xyz][i+1][j+1][t+1]*(pi/pairAB.exp[ii])^0.5
            end
        end
    end

    ii = 0
    for jprim in 1:size(pairAB.g_b.exp)[1], iprim in 1:size(pairAB.g_a.exp)[1]
        ii += 1
        i=0
        j=0
        T[ii][i+1][j+1] = (pairAB.g_a.exp[iprim] - 2*pairAB.g_a.exp[iprim]^2 * ((pairAB.coord[ii][xyz]-pairAB.g_a.coord[xyz])^2 + 1/(2*pairAB.exp[ii]))) * S[ii][i+1][j+1]
        for j in 1:lb
            T[ii][i+1][j+1] = (pairAB.coord[ii][xyz]-pairAB.g_b.coord[xyz])*T[ii][i+1][j+1] + pairAB.g_a.exp[iprim]/pairAB.exp[ii]*2*pairAB.g_b.exp[jprim]*S[ii][i+1][j+1] 
            if(j>1)
                T[ii][i+1][j+1] += (j-1)/(2*pairAB.exp[ii])*T[ii][i+1][j-1] - pairAB.g_a.exp[iprim]/pairAB.exp[ii]*(j)*S[ii][i+1][j-1]
            end
        end
        for i in 1:la
            for j in max(0,lb-la+i):lb
                T[ii][i+1][j+1] = (pairAB.coord[ii][xyz]-pairAB.g_a.coord[xyz])*T[ii][i][j+1] + pairAB.g_b.exp[jprim]/pairAB.exp[ii]*2*pairAB.g_a.exp[iprim]*S[ii][i+1][j+1] 
                if(i>1)
                    T[ii][i+1][j+1] += (i-1)/(2*pairAB.exp[ii])*T[ii][i-1][j+1] - pairAB.g_b.exp[jprim]/pairAB.exp[ii]*(i-1)*S[ii][i-1][j+1]
                end
                if(j>0)
                    T[ii][i+1][j+1] += (j)/(2*pairAB.exp[ii])*T[ii][i][j]# - pairAB.g_b.exp[jprim]/pairAB.exp[ii]*(i-1)*S[ii][i-1][j+1]
                end
            end
        end
    end       
        
    return [T[ii][la+1][lb+1] for ii in eachindex(pairAB.exp)]
        
end

Tij (generic function with 1 method)

In [22]:
function kinetic(pairAB)
    
    g_a = pairAB.g_a
    g_b = pairAB.g_b

    Eab = build_E(g_a.l,g_b.l,pairAB)

    result = zeros((size(g_a.orientaciones)[1],size(g_b.orientaciones)[1]))

    for (a,(lax,lay,laz)) in enumerate(g_a.orientaciones)
        for (b,(lbx,lby,lbz)) in enumerate(g_b.orientaciones)
        
            Tx = Tij(lax,lbx,1,pairAB)
            Ty = Tij(lay,lby,2,pairAB)
            Tz = Tij(laz,lbz,3,pairAB)
        
            integral = 0.0
            for ii in eachindex(pairAB.exp)
                tx,ty,tz = 0,0,0
                Sx = Eab[ii][1][lax+1][lbx+1][tx+1]*(pi/pairAB.exp[ii])^0.5
                Sy = Eab[ii][2][lay+1][lby+1][ty+1]*(pi/pairAB.exp[ii])^0.5
                Sz = Eab[ii][3][laz+1][lbz+1][tz+1]*(pi/pairAB.exp[ii])^0.5
                integral += pairAB.coef[ii]*(Tx[ii]*Sy*Sz + Ty[ii]*Sx*Sz + Tz[ii]*Sx*Sy)
            end           
            result[a,b] = g_a.N[a]*g_b.N[b]*integral
        end
    end
    
    return result
    
end

kinetic (generic function with 1 method)

In [23]:
pair = build_shell_pair(shells[1],shells[1])
results = kinetic(pair)

LoadError: BoundsError: attempt to access 0-element Vector{Any} at index [1]

In [24]:
function build_T(shells)
    
    nbf = get_nbf(shells)
    
    T = zeros(nbf,nbf)
    i = 1
    for g_a in shells
        j = 1
        for g_b in shells
            pair = build_shell_pair(g_a,g_b)
            results = kinetic(pair)
            T[i:i+size(g_a.orientaciones)[1]-1,j:j+size(g_b.orientaciones)[1]-1] = results
            j += size(g_b.orientaciones)[1]
        end
        i += size(g_a.orientaciones)[1]
    end
    
    return T
    
end

build_T (generic function with 1 method)

In [25]:
@btime T = build_T(shells)

  49.873 ns (1 allocation: 64 bytes)


0×0 Matrix{Float64}

# Potential

In [26]:
function build_R_one(coord,pairAB)
    
    p = pairAB.exp
    Rpc = [pairAB.coord[ii]-coord for ii in eachindex(pairAB.exp)]
    F = [[F_n(n,p[ii]*norm(Rpc[ii])^2) for n in 0:pairAB.g_a.l + pairAB.g_b.l] for ii in eachindex(pairAB.exp)]
    lt = pairAB.g_a.l + pairAB.g_b.l
    R = [[[[[0.0 for lz in 0:lt-n-lx-ly] for ly in 0:lt-n-lx] for lx in 0:lt-n] for n in 0:lt] for ii in eachindex(pairAB.exp)]

    for ii in eachindex(pairAB.exp)
        for nini in 0:lt
            R[ii][nini+1][1][1][1] = (-2*p[ii])^nini*2*pi/p[ii]*F[ii][nini+1]
            l = 0
            for n in nini-1:-1:0
                l += 1
                for lx in 0:l
                    for ly in 0:l-lx
                        lz = l-lx-ly
                        if(lz>=2)
                            R[ii][n+1][lx+1][ly+1][lz+1] += (lz-1)*R[ii][n+2][lx+1][ly+1][lz-1]
                        end
                        if(lz>=1)
                            R[ii][n+1][lx+1][ly+1][lz+1] += Rpc[ii][3]*R[ii][n+2][lx+1][ly+1][lz]
                        end
                    end
                    ly = l-lx
                    if(ly>=2)
                        R[ii][n+1][lx+1][l-lx+1][1] += (ly-1)*R[ii][n+2][lx+1][l-lx-1][1]
                    end
                    if(ly>=1)
                        R[ii][n+1][lx+1][l-lx+1][1] += Rpc[ii][2]*R[ii][n+2][lx+1][l-lx][1]
                    end
                end
                lx = l
                if(lx>=2)
                    R[ii][n+1][l+1][1][1] += (lx-1)*R[ii][n+2][l-1][1][1]
                end
                if(lx>=1)
                    R[ii][n+1][l+1][1][1] += Rpc[ii][1]*R[ii][n+2][l][1][1]                
                end
            end
        end
    end
             
    return R

end

build_R_one (generic function with 1 method)

In [27]:
function potential(pairAB,Z,coord)

    g_a = pairAB.g_a
    g_b = pairAB.g_b
    
    R = build_R_one(coord,pairAB)
    Eab = build_E(pairAB.g_a.l,pairAB.g_b.l,pairAB)
    
    result = zeros(size(g_a.orientaciones)[1],size(g_b.orientaciones)[1])

    for (a,(lax,lay,laz)) in enumerate(g_a.orientaciones)
        for (b,(lbx,lby,lbz)) in enumerate(g_b.orientaciones)
        
            integral = 0.0
            for ii in eachindex(pairAB.exp)
                aux3 = 0
                for t_z in 0:laz+lbz
                    aux2 = 0    
                    for t_y in 0:lay+lby
                        aux1 = 0       
                        for t_x in 0:lax+lbx
                            aux1 += Eab[ii][1][lax+1][lbx+1][t_x+1]*R[ii][1][t_x+1][t_y+1][t_z+1]
                        end
                        aux2 += Eab[ii][2][lay+1][lby+1][t_y+1]*aux1
                    end
                    aux3 += Eab[ii][3][laz+1][lbz+1][t_z+1]*aux2
                end
                integral += aux3*pairAB.coef[ii]
            end           
            result[a,b] = g_a.N[a]*g_b.N[b]*integral
        end
    end
    
    return Z*result
    
end

potential (generic function with 1 method)

In [28]:
function build_V(shells,Zs,coords)
    
    nbf = get_nbf(shells)
    
    V = zeros(nbf,nbf)
    i = 1
    for g_a in shells
        j = 1
        for g_b in shells
            pair = build_shell_pair(g_a,g_b)
            for (Z,coord) in zip(Zs,coords)
                results = potential(pair,Z,coord)
                V[i:i+size(g_a.orientaciones)[1]-1,j:j+size(g_b.orientaciones)[1]-1] += results
            end
            j += size(g_b.orientaciones)[1]
        end
        i += size(g_a.orientaciones)[1]
    end
    
    return V
    
end

build_V (generic function with 1 method)

In [29]:
@btime V = build_V(shells,mol.Zs,mol.coords)

  103.036 ns (3 allocations: 160 bytes)


0×0 Matrix{Float64}

## 4-C ERIs

In [30]:
function ERI(pairAB,pairCD)

    g_a = pairAB.g_a
    g_b = pairAB.g_b
    g_c = pairCD.g_a
    g_d = pairCD.g_b
    
    lt = pairAB.g_a.l + pairAB.g_b.l + pairCD.g_a.l + pairCD.g_b.l

    # Build boys
    alpha = [[pairAB.exp[ii]*pairCD.exp[jj]/(pairAB.exp[ii]+pairCD.exp[jj]) for jj in eachindex(pairCD.exp)] for ii in eachindex(pairAB.exp)]
    Rpq = [[pairAB.coord[ii]-pairCD.coord[jj] for jj in eachindex(pairCD.exp)] for ii in eachindex(pairAB.exp)]
    F = [[[F_n(n,alpha[ii][jj]*norm(Rpq[ii][jj])^2) for n in 0:lt] for jj in eachindex(pairCD.exp)] for ii in eachindex(pairAB.exp)]

    # Build R
    R = build_R(lt,alpha,Rpq,F,pairAB,pairCD)

    # Build E
    Eab = build_E(pairAB.g_a.l,pairAB.g_b.l,pairAB)
    Ecd = build_E(pairCD.g_a.l,pairCD.g_b.l,pairCD)

    result = zeros((size(g_a.orientaciones)[1],size(g_b.orientaciones)[1],size(g_c.orientaciones)[1],size(g_d.orientaciones)[1]))
    for (a,(lax,lay,laz)) in enumerate(g_a.orientaciones)
        for (b,(lbx,lby,lbz)) in enumerate(g_b.orientaciones)
            for (c,(lcx,lcy,lcz)) in enumerate(g_c.orientaciones)
                for (d,(ldx,ldy,ldz)) in enumerate(g_d.orientaciones)

                    integral = 0.0
                    for ii in eachindex(pairAB.exp)
                        aux7 = 0
                        for t_z in 0:laz+lbz
                            aux6 = 0    
                            for t_y in 0:lay+lby
                                aux5 = 0        
                                for t_x in 0:lax+lbx
                                    aux4 = 0
                                    for jj in eachindex(pairCD.exp)
                                        aux3 = 0
                                        for u_z in 0:lcz+ldz
                                            aux2 = 0
                                            for u_y in 0:lcy+ldy
                                                aux1 = 0
                                                for u_x in 0:lcx+ldx
                                                    aux1 += (-1)^(u_x+u_y+u_z)*Ecd[jj][1][lcx+1][ldx+1][u_x+1]*R[ii][jj][1][t_x+u_x+1][t_y+u_y+1][t_z+u_z+1]
                                                end
                                                aux2 += Ecd[jj][2][lcy+1][ldy+1][u_y+1]*aux1
                                            end
                                            aux3 += Ecd[jj][3][lcz+1][ldz+1][u_z+1]*aux2
                                        end
                                        aux4 += aux3*pairCD.coef[jj]
                                    end
                                    aux5 += Eab[ii][1][lax+1][lbx+1][t_x+1]*aux4
                                end
                                aux6 += Eab[ii][2][lay+1][lby+1][t_y+1]*aux5
                            end
                            aux7 += Eab[ii][3][laz+1][lbz+1][t_z+1]*aux6
                        end
                        integral += aux7*pairAB.coef[ii]
                    end           
                    result[a,b,c,d] = g_a.N[a]*g_b.N[b]*g_c.N[c]*g_d.N[d]*integral
                end
            end
        end
    end
    
    return result
    
end      

ERI (generic function with 1 method)

In [31]:
@btime ERI(pairAB,pairCD)

  6.955 μs (138 allocations: 10.11 KiB)


1×3×1×1 Array{Float64, 4}:
[:, :, 1, 1] =
 0.008391  -0.00329528  -0.0099135

In [32]:
function build_I4(shells)
    
    nbf = get_nbf(shells)
    
    I = zeros(nbf,nbf,nbf,nbf)
    i = 1
    for g_a in shells
        j = 1
        for g_b in shells
            pairAB = build_shell_pair(g_a,g_b)
            k = 1
            for g_c in shells
                l = 1
                for g_d in shells
                    pairCD = build_shell_pair(g_c,g_d)
                    results = ERI(pairAB,pairCD)
                    I[i:i+size(g_a.orientaciones)[1]-1,
                      j:j+size(g_b.orientaciones)[1]-1,
                      k:k+size(g_c.orientaciones)[1]-1,
                      l:l+size(g_d.orientaciones)[1]-1] = results
                    l += size(g_d.orientaciones)[1]
                end
                k += size(g_c.orientaciones)[1]
            end
            j += size(g_b.orientaciones)[1]
        end
        i += size(g_a.orientaciones)[1]
    end
    
    return I
    
end

build_I4 (generic function with 1 method)

In [33]:
@btime I = build_I4(shells)

  12.538 s (104074729 allocations: 7.39 GiB)


43×43×43×43 Array{Float64, 4}:
[:, :, 1, 1] =
  4.74158    -0.475541   0.594306  …  -0.182563    0.0716955   0.215688
 -0.475541    1.13425    0.829567     -0.292964    0.115051    0.34612
  0.594306    0.829567   0.868217     -0.311596    0.122369    0.368133
  0.223358    0.449967   0.504971     -0.181305    0.0712012   0.214201
  0.0         0.0        0.0           0.173768    0.0293387   0.0882622
  0.0         0.0        0.0       …   0.0293387   0.236953   -0.034662
  0.0         0.0        0.0           0.0882622  -0.034662    0.144198
  0.0         0.0        0.0           0.219074    0.0317269   0.095447
  0.0         0.0        0.0           0.0317269   0.287403   -0.0374835
  0.0         0.0        0.0           0.095447   -0.0374835   0.187097
  0.0         0.0        0.0       …   0.196952    0.0155876   0.0468936
  0.0         0.0        0.0           0.0155876   0.230523   -0.0184159
  0.0         0.0        0.0           0.0468936  -0.0184159   0.181242
  ⋮            

## 2C ERIs

In [34]:
function normalize_aux_shell(g_a)
    
    g_1 = build_zero_shell()
    
    pairAB = build_shell_pair(g_a,g_1)
    results = ERI(pairAB,pairAB)

    N = diag(results[1:size(g_a.orientaciones)[1],1,1:size(g_a.orientaciones)[1],1])
    N = 1.0 ./ (sqrt.(N))
    
    return N
    
    l = g_a.l
    orientaciones = []
    for lx in l:-1:0
        for ly in l-lx:-1:0
            lz = l-lx-ly
            append!(orientaciones,[[lx,ly,lz]])      
        end
    end        
    
    aux_normalized_shell = shell(g_a.l,g_a.exp,g_a.coef,g_a.coord,N,orientaciones)
    
    return aux_normalized_shell
    
end

function build_aux_shell(l,exp,coef,coord)

    orientaciones = []
    for lx in l:-1:0
        for ly in l-lx:-1:0
            lz = l-lx-ly
            append!(orientaciones,[[lx,ly,lz]])      
        end
    end        

    N = ones(Int((l+1)*(l+2)/2))

    tmp_shell = shell(l, exp, coef, coord, N, orientaciones)

    N = normalize_aux_shell(tmp_shell)    
    
    return shell(l, exp, coef, coord, N, orientaciones)
    
end

build_aux_shell (generic function with 1 method)

In [35]:
shells_aux = build_basis(mol,"aug-cc-pvdz-jkfit",normalized=false,auxiliar=true)

52-element Vector{Any}:
 shell(0, [1517.8667506], [1.0], [0.0, 0.0, 0.0], [1904.7905151311072], [[0, 0, 0]])
 shell(0, [489.67952008], [1.0], [0.0, 0.0, 0.0], [463.12165258630614], [[0, 0, 0]])
 shell(0, [176.72118665], [1.0], [0.0, 0.0, 0.0], [129.54359170558996], [[0, 0, 0]])
 shell(0, [63.79223314], [1.0], [0.0, 0.0, 0.0], [36.24640319610661], [[0, 0, 0]])
 shell(0, [25.36649913], [1.0], [0.0, 0.0, 0.0], [11.445396102474305], [[0, 0, 0]])
 shell(0, [9.91354912], [1.0], [0.0, 0.0, 0.0], [3.536646181981861], [[0, 0, 0]])
 shell(0, [4.46453066], [1.0], [0.0, 0.0, 0.0], [1.304742840640154], [[0, 0, 0]])
 shell(0, [1.80177437], [1.0], [0.0, 0.0, 0.0], [0.41969197615897513], [[0, 0, 0]])
 shell(0, [0.80789711], [1.0], [0.0, 0.0, 0.0], [0.1539927336073522], [[0, 0, 0]])
 shell(0, [0.33864327], [1.0], [0.0, 0.0, 0.0], [0.05193776076506855], [[0, 0, 0]])
 shell(1, [120.16030921], [1.0], [0.0, 0.0, 0.0], [3037.2294228145483, 3037.2294228145483, 3037.2294228145483], [[1, 0, 0], [0, 1, 0], [0, 

In [36]:
function build_I2(shells_aux)
    
    g_1 = build_zero_shell()
    
    nbfaux = get_nbf(shells_aux)
    
    I = zeros(nbfaux,nbfaux)
    i = 1
    for g_a in shells_aux
        pairAB = build_shell_pair(g_a,g_1)
        j = 1
        for g_b in shells_aux
            pairCD = build_shell_pair(g_b,g_1)
            results = ERI(pairAB,pairCD)
            I[i:i+size(g_a.orientaciones)[1]-1,
              j:j+size(g_b.orientaciones)[1]-1] = results[1:size(g_a.orientaciones)[1],1,1:size(g_b.orientaciones)[1],1]
            j += size(g_b.orientaciones)[1]
        end
        i += size(g_a.orientaciones)[1]
    end
    
    return I
    
end

build_I2 (generic function with 1 method)

In [37]:
@btime I = build_I2(shells_aux)

  22.714 ms (271768 allocations: 18.13 MiB)


171×171 Matrix{Float64}:
  1.0         0.926763    0.781832   …   0.0222496     0.116471
  0.926763    1.0         0.93961        0.0295045     0.154529
  0.781832    0.93961     1.0            0.0380062     0.199328
  0.627276    0.799166    0.939636       0.0488176     0.256997
  0.504281    0.657862    0.814015       0.0608406     0.323158
  0.400729    0.528131    0.669728   …   0.0749227     0.40718
  0.328861    0.435021    0.556826       0.0867681     0.493398
  0.262346    0.347668    0.447111       0.0949723     0.607518
  0.214748    0.284786    0.366895       0.0894287     0.716902
  0.17282     0.229257    0.295606       0.0636553     0.827662
  0.0         0.0         0.0        …   0.0022558     0.00422074
  0.0         0.0         0.0            0.0137826    -0.00165755
  0.0         0.0         0.0            0.00221075    0.00227244
  ⋮                                  ⋱                 ⋮
  0.0339095   0.0449453   0.057825       0.663561      0.0
  0.131082    0.173894

## 3C ERIs

In [38]:
function build_I3(shells,shells_aux)
    
    g_1 = build_zero_shell()
    
    nbf = get_nbf(shells)
    nbfaux = get_nbf(shells_aux)
    
    I = zeros(nbf,nbf,nbfaux)
    i = 1
    for g_a in shells
        j = 1
        for g_b in shells
            pairAB = build_shell_pair(g_a,g_b)
            k = 1
            for g_c in shells_aux
                pairCD = build_shell_pair(g_c,g_1)
                results = ERI(pairAB,pairCD)
                I[i:i+size(g_a.orientaciones)[1]-1,
                  j:j+size(g_b.orientaciones)[1]-1,
                  k:k+size(g_c.orientaciones)[1]-1] = results[1:size(g_a.orientaciones)[1],1:size(g_b.orientaciones)[1],1:size(g_c.orientaciones)[1],1]
                k += size(g_c.orientaciones)[1]
            end
            j += size(g_b.orientaciones)[1]
        end
        i += size(g_a.orientaciones)[1]
    end
    
    return I
    
end

build_I3 (generic function with 1 method)

In [39]:
@btime I = build_I3(shells,shells_aux)

  522.760 ms (5059692 allocations: 354.16 MiB)


43×43×171 Array{Float64, 3}:
[:, :, 1] =
  1.32971    -0.184683    0.130411   …   0.015698     0.0472257
 -0.184683    0.224804    0.14441        0.0201141    0.0605111
  0.130411    0.14441     0.157335       0.0221406    0.0666075
  0.0487962   0.0791035   0.0911653      0.0128416    0.0386325
  0.0         0.0         0.0            0.00527007   0.0158544
  0.0         0.0         0.0        …   0.0425512   -0.00622629
  0.0         0.0         0.0           -0.00622629   0.0258897
  0.0         0.0         0.0            0.00569132   0.0171217
  0.0         0.0         0.0            0.0515535   -0.00672397
  0.0         0.0         0.0           -0.00672397   0.0335603
  0.0         0.0         0.0        …   0.00279592   0.00841122
  0.0         0.0         0.0            0.0413472   -0.00330322
  0.0         0.0         0.0           -0.00330322   0.0325078
  ⋮                                  ⋱               
 -0.0182737  -0.0234144  -0.0257734      0.00283102   0.000941481
 -0